In [19]:
import numpy as np
import sounddevice as sd
import soundfile as sf
import matplotlib.pyplot as plt

SAMPLE_RATE = 44100
BIT_DURATION = 1
FREQ_LOW = 440
FREQ_HIGH = 880

def generate_tone(frequency, duration, sample_rate=SAMPLE_RATE):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    tone = np.sin(2 * np.pi * frequency * t)
    window = np.hanning(len(tone))
    return tone * window

In [20]:
def encode_nrz(data_bits):
    audio_signal = np.array([])
    for bit in data_bits:
        freq = FREQ_HIGH if bit == '1' else FREQ_LOW
        tone = generate_tone(freq, BIT_DURATION)
        audio_signal = np.concatenate([audio_signal, tone])
    return audio_signal

def encode_manchester(data_bits):
    audio_signal = np.array([])
    for bit in data_bits:
        if bit == '1':
            tone1 = generate_tone(FREQ_HIGH, BIT_DURATION / 2)
            tone2 = generate_tone(FREQ_LOW, BIT_DURATION / 2)
        else:
            tone1 = generate_tone(FREQ_LOW, BIT_DURATION / 2)
            tone2 = generate_tone(FREQ_HIGH, BIT_DURATION / 2)
        audio_signal = np.concatenate([audio_signal, tone1, tone2])
    return audio_signal


In [21]:
def detect_frequency(audio_segment, sample_rate=SAMPLE_RATE):
    fft_result = np.fft.fft(audio_segment)
    freqs = np.fft.fftfreq(len(audio_segment), d=1/sample_rate)
    magnitude = np.abs(fft_result[:len(fft_result)//2])
    freqs_positive = freqs[:len(freqs)//2]
    peak_idx = np.argmax(magnitude)
    return abs(freqs_positive[peak_idx])

def frequency_to_bit(frequency, threshold=660):
    return '1' if frequency > threshold else '0'

def decode_nrz(audio_signal, num_bits):
    samples_per_bit = int(SAMPLE_RATE * BIT_DURATION)
    decoded_bits = ""
    for i in range(num_bits):
        start_idx = i * samples_per_bit
        end_idx = start_idx + samples_per_bit
        if end_idx > len(audio_signal):
            break
        segment = audio_signal[start_idx:end_idx]
        freq = detect_frequency(segment)
        decoded_bits += frequency_to_bit(freq)
    return decoded_bits

def decode_manchester(audio_signal, num_bits):
    samples_per_bit = int(SAMPLE_RATE * BIT_DURATION)
    decoded_bits = ""
    for i in range(num_bits):
        start_idx = i * samples_per_bit
        end_idx = start_idx + samples_per_bit
        if end_idx > len(audio_signal):
            break
        mid = start_idx + samples_per_bit // 2
        first_half = audio_signal[start_idx + samples_per_bit//8 : mid - samples_per_bit//8]
        second_half = audio_signal[mid + samples_per_bit//8 : end_idx - samples_per_bit//8]
        freq1 = detect_frequency(first_half)
        freq2 = detect_frequency(second_half)
        state1 = frequency_to_bit(freq1)
        state2 = frequency_to_bit(freq2)
        if state1 == '1' and state2 == '0':
            decoded_bits += '1'
        elif state1 == '0' and state2 == '1':
            decoded_bits += '0'
        else:
            decoded_bits += '?'  # erro
    return decoded_bits


In [23]:
def detect_frequency(audio_segment, sample_rate=SAMPLE_RATE):
    fft_result = np.fft.fft(audio_segment)
    freqs = np.fft.fftfreq(len(audio_segment), d=1/sample_rate)
    magnitude = np.abs(fft_result[:len(fft_result)//2])
    freqs_positive = freqs[:len(freqs)//2]
    peak_idx = np.argmax(magnitude)
    return abs(freqs_positive[peak_idx])

def frequency_to_bit(frequency, threshold=660):
    return '1' if frequency > threshold else '0'

def decode_nrz(audio_signal, num_bits):
    samples_per_bit = int(SAMPLE_RATE * BIT_DURATION)
    decoded_bits = ""
    for i in range(num_bits):
        start_idx = i * samples_per_bit
        end_idx = start_idx + samples_per_bit
        if end_idx > len(audio_signal):
            break
        segment = audio_signal[start_idx:end_idx]
        freq = detect_frequency(segment)
        decoded_bits += frequency_to_bit(freq)
    return decoded_bits

def decode_manchester(audio_signal, num_bits):
    samples_per_bit = int(SAMPLE_RATE * BIT_DURATION)
    decoded_bits = ""
    for i in range(num_bits):
        start_idx = i * samples_per_bit
        end_idx = start_idx + samples_per_bit
        if end_idx > len(audio_signal):
            break
        mid = start_idx + samples_per_bit // 2
        first_half = audio_signal[start_idx + samples_per_bit//8 : mid - samples_per_bit//8]
        second_half = audio_signal[mid + samples_per_bit//8 : end_idx - samples_per_bit//8]
        freq1 = detect_frequency(first_half)
        freq2 = detect_frequency(second_half)
        state1 = frequency_to_bit(freq1)
        state2 = frequency_to_bit(freq2)
        if state1 == '1' and state2 == '0':
            decoded_bits += '1'
        elif state1 == '0' and state2 == '1':
            decoded_bits += '0'
        else:
            decoded_bits += '?'  # erro
    return decoded_bits


In [24]:
# Caminho do arquivo de áudio (que foi enviado para o celular)
arquivo_alvo = 'dados_ar.wav'  # <-- Substitua pelo nome correto se for diferente

# Quantidade de bits (você precisa saber o que foi enviado!)
num_bits_estimado = 8  # exemplo: "00111000"

# Leitura do áudio
audio_testado, sr = sf.read(arquivo_alvo)

# Tente decodificar com o tipo de modulação usado
decoded = decode_nrz(audio_testado, num_bits_estimado)
# decoded = decode_manchester(audio_testado, num_bits_estimado)

# Resultado
print("Decodificação do arquivo:")
print(f"Bits decodificados: {decoded}")

Decodificação do arquivo:
Bits decodificados: 11111
